<a href="https://colab.research.google.com/github/darshan-kale-dsi/Team-05-Bank-Marketing/blob/seb_branch/notebooks/model_adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Adjustment

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
dataframe = pd.read_csv('bank-full.csv', sep = ';')

In [3]:
# trimming extreme values (outliers)
dataframe = dataframe[(dataframe['balance']<10000) & (dataframe['duration']<1800)]
dataframe['pdays'] = dataframe['pdays'].apply(lambda value: 0 if value==-1 else 1 )
dataframe['default'] = dataframe['default'].apply(lambda value: 1 if value =='yes' else 0)
dataframe['housing'] = dataframe['housing'].apply(lambda value: 1 if value =='yes' else 0)
dataframe['loan'] = dataframe['loan'].apply(lambda value: 1 if value =='yes' else 0)
dataframe['y']= dataframe['y'].apply(lambda value: 1 if value =='yes' else 0)
dataframe['month'] = dataframe['month'].replace({'may':5,
                            'jun':6,
                            'jul':7,
                            'aug':8,
                            'oct':10,
                            'nov':11,
                            'dec':12,
                            'jan':1,
                            'feb':2,
                            'mar':3,
                            'apr':4,
                            'sep':9})

<ipython-input-3-bcfe183c4271>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe['month'] = dataframe['month'].replace({'may':5,


In [4]:
import numpy as np

dataframe['day_sine'] = dataframe['day'].apply(lambda value: np.sin(2*value*np.pi/31))
dataframe['day_cosine'] = dataframe['day'].apply(lambda value: np.cos(2*value*np.pi/31))

dataframe['month_sine'] = dataframe['month'].apply(lambda value: np.sin(2*value*np.pi/12))
dataframe['month_cosine'] = dataframe['month'].apply(lambda value: np.cos(2*value*np.pi/12))

In [5]:
dataframe = dataframe.drop(['day', 'month'], axis = 1)

In [6]:
X = dataframe.drop('y', axis = 1)
y = dataframe['y']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 123)

In [7]:
# def preprocessing_pipeline(dataframe):
#   binary_features = [
#                       'pdays',
#                       'default',
#                       'housing',
#                       'loan'
#                     ]
#   numeric_features = [
#                       'age',
#                       'balance',
#                       # 'duration',
#                       'campaign',
#                       'previous'
#                       ]
#   time_features = [
#                     'day_sine',
#                    'day_cosine',
#                    'month_sine',
#                    'month_cosine'
#                    ]
#   categorical_features = [
#                           'job',
#                           'marital',
#                           'education',
#                           'contact',
#                           'poutcome'
#                           ]

#   dataframe = dataframe[binary_features+numeric_features+time_features+categorical_features]

#   preprocessor = ColumnTransformer(transformers=[
#       ('num', StandardScaler(), numeric_features),
#       ('cat', OneHotEncoder(), categorical_features)
#   ])

#   dataframe_transformed = preprocessor.fit_transform(dataframe)

#   ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
#   final_columns = numeric_features + list(ohe_feature_names)
#   X_train_preprocessed_df = pd.DataFrame(dataframe_transformed, columns=final_columns)

#   X_train_preprocessed_df = pd.concat([X_train_preprocessed_df,dataframe[time_features].reset_index(drop = True)], axis = 1)
#   X_train_preprocessed_df = pd.concat([X_train_preprocessed_df,dataframe[binary_features].reset_index(drop = True)], axis = 1)


#   return X_train_preprocessed_df

In [8]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocessing_pipeline(df):
    """
    Preprocess the input DataFrame by scaling numeric features, one-hot encoding categorical features,
    and preserving time and binary features unchanged.

    Parameters
    ----------
    df : pandas.DataFrame
        The input DataFrame containing all necessary columns.

    Returns
    -------
    df_preprocessed : pandas.DataFrame
        A DataFrame with transformed numeric and categorical features, along with the original time and binary features.
        The columns are ordered as: [scaled numeric features, one-hot encoded categorical features, time features, binary features].
    """

    # Define feature groups
    binary_features = [
                      # 'pdays',
                      'default',
                      'housing',
                      'loan'
                    ]
    numeric_features = [
                      'age',
                      'balance',
                      # 'duration',
                      'campaign',
                      'previous'
                      ]
    time_features = [
                    'day_sine',
                   'day_cosine',
                   'month_sine',
                   'month_cosine'
                   ]
    categorical_features = [
                          'job',
                          'marital',
                          'education',
                          'contact',
                          # 'poutcome'
                          ]

    # Select and copy only the necessary columns
    selected_columns = binary_features + numeric_features + time_features + categorical_features
    print(selected_columns)
    df = df[selected_columns].copy()

    # Create a ColumnTransformer to scale numeric features and one-hot encode categorical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(), categorical_features)
        ]
    )

    # Apply transformations on numeric and categorical features
    transformed_array = preprocessor.fit_transform(df)

    # Retrieve one-hot encoded feature names
    cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)

    # Build DataFrame from the transformed numeric and categorical features
    transformed_feature_names = numeric_features + list(cat_feature_names)
    df_transformed = pd.DataFrame(transformed_array, columns=transformed_feature_names, index=df.index)

    # Concatenate the unchanged time and binary features
    df_preprocessed = pd.concat([df_transformed, df[time_features], df[binary_features]], axis=1)

    return df_preprocessed


In [9]:
X_train_processed = preprocessing_pipeline(X_train)
X_test_processed = preprocessing_pipeline(X_test)



['default', 'housing', 'loan', 'age', 'balance', 'campaign', 'previous', 'day_sine', 'day_cosine', 'month_sine', 'month_cosine', 'job', 'marital', 'education', 'contact']
here
here2
['default', 'housing', 'loan', 'age', 'balance', 'campaign', 'previous', 'day_sine', 'day_cosine', 'month_sine', 'month_cosine', 'job', 'marital', 'education', 'contact']
here
here2


In [10]:
from sklearn import ensemble as en
import boruta as bt
x_train_n = X_train_processed.values
y_train_n = y_train.values.ravel()

# Define model
et = en.ExtraTreesClassifier( n_estimators=250, random_state=0, n_jobs=-1 )

# Define boruta
boruta = bt.BorutaPy( et, n_estimators='auto', verbose=2, random_state=42 ).fit( x_train_n, y_train_n  )

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	7
Tentative: 	0
Rejected: 	26


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	0
Rejected: 	26


In [11]:
cols_selected = boruta.support_.tolist()
# X_train_processed = X_train_processed[X_train_processed.iloc[:, cols_selected].columns.to_list()]
# X_test_processed = X_test_processed[X_train_processed.columns]

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

# model = BernoulliNB()
# model = KNeighborsClassifier()
# model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
# model = RandomForestClassifier()
model = XGBClassifier()
# model = XGBClassifier(colsample_bytree = 0.7,
#                       gamma = 0.2,
#                       learning_rate = 0.1,
#                       max_depth = 18,
#                       reg_alpha = 0.1,
#                       reg_lambda = 100)
# model = LogisticRegression()
# model = svm.SVC()
model.fit(X_train_processed, y_train)
y_hat = model.predict(X_test_processed)
print(accuracy_score(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))

0.9711661522246481
[[12519   278]
 [  105   381]]


In [13]:
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

# Space
space = {
    'learning_rate': hp.choice('learning_rate', [0.0001,0.001, 0.01, 0.1, 1]),
    'max_depth' : hp.choice('max_depth', range(3,21,3)),
    'gamma' : hp.choice('gamma', [i/10.0 for i in range(0,5)]),
    'colsample_bytree' : hp.choice('colsample_bytree', [i/10.0 for i in range(3,10)]),
    'reg_alpha' : hp.choice('reg_alpha', [1e-5, 1e-2, 0.1, 1, 10, 100]),
    'reg_lambda' : hp.choice('reg_lambda', [1e-5, 1e-2, 0.1, 1, 10, 100])
}

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)


# Objective function
def objective(params):
  xgboost = XGBClassifier(seed=0, **params)
  score = cross_val_score(estimator=xgboost,
                            X=X_train_processed,
                            y=y_train,
                            cv=kfold,
                            scoring='average_precision',
                            n_jobs=-1).mean()
  # Loss is negative score
  loss = - score
  # Dictionary with information for evaluation
  return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 48, trials = Trials())

# Print the values of the best parameters
print(space_eval(space, best))

100%|██████████| 48/48 [02:15<00:00,  2.83s/trial, best loss: -0.3910000953458241]
{'colsample_bytree': 0.5, 'gamma': 0.3, 'learning_rate': 0.01, 'max_depth': 18, 'reg_alpha': 1e-05, 'reg_lambda': 1}


In [14]:


# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.1, 0.01, 0.001],
    "subsample": [0.8, 1.0],
    # "colsample_bytree": [0.8, 1.0],
    # "min_child_weight": [1, 3, 5],
    # "gamma": [0, 1, 5],
    # "reg_alpha": [0, 1, 10],
    # "reg_lambda": [0, 1, 10],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator = model,
    param_grid = param_grid,
    scoring = "roc_auc",
    cv = 5,
    verbose = 1,
    n_jobs = -1
)

# Fit the grid search
final = grid_search.fit(X_train_processed, y_train)
y_test = final.predict(X_test_processed)
print(accuracy_score(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
0.990589475269141
[[12756    41]
 [   84   402]]
Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 1.0}
Best cross-validation score: 0.7936587791330002
